In [1]:
import pandas as pd
import numpy as np

def combine_wiki_rranks_popularities(csz, save_file):    
    sub_rr = pd.read_csv('/data/khodadaa/lucene-index/rr/05092047_sub2_rranks.csv', header=None) 
    all_rr = pd.read_csv('/data/khodadaa/lucene-index/rranks.csv', header=None)
    sub_rr = sub_rr[[0,3]]
    all_rr = all_rr[[0,3]]
    sub_rr.rename(columns={0: 'Query', 3: csz}, inplace=True)
    all_rr.rename(columns={0: 'Query', 3: '100'}, inplace=True)
    pop_df = pd.read_csv('/data/khodadaa/analysis/msn_inner.csv', usecols=['query', 'popularity'])
    pop_df.rename(columns={'query': 'Query', 'popularity': 'TestViewCount'}, inplace=True)
    pop_df = pop_df.loc[pop_df.groupby(['Query'])['TestViewCount'].idxmax()]
    rr = pd.merge(sub_rr, all_rr, on='Query', how='inner', validate='1:1')
    mg = pd.merge(pop_df, rr, on='Query', how='inner', validate='1:1')
    mg.to_csv(save_file, index=False)

# For stackoverflow uncomment following lines
# csz = '18'
# rranks_file = '/data/khodadaa/analysis/stack.csv'
# features_file = "/data/khodadaa/stack_results/stack_feat/{}.csv".format(csz)
# features_complement_file = "/data/khodadaa/stack_results/stack_feat/c{}.csv".format(csz)
# ml_file = "/data/khodadaa/stack_results/stack_feat/ml_in_{}.csv".format(csz)


# For wikipedia uncomment following lines
csz = '2'
rranks_file = '/data/khodadaa/analysis/wiki.csv'
features_file = "/data/khodadaa/stack_results/wiki/msn_{}.csv".format(csz)
features_complement_file = "/data/khodadaa/stack_results/wiki/msn_c{}.csv".format(csz)
ml_file = "/data/khodadaa/stack_results/wiki/ml_in_{}.csv".format(csz)
combine_wiki_rranks_popularities(csz, rranks_file)

In [2]:
%%time
feat_df = pd.read_csv(features_file)
feat_complement_df = pd.read_csv(features_complement_file)

# drop last column which is blank
feat_df.drop(columns=feat_df.columns[-1:], inplace=True)
feat_complement_df.drop(columns=feat_complement_df.columns[-1:], inplace=True)

rr_df = pd.read_csv(rranks_file).filter(items=['Query', 'TestViewCount', csz, '100'])
# rr_df.rename(columns={csz: 'sub_rr', '100': 'all_rr', 'ql_t': 'sub_ql', 'ql_t.1': 'all_ql'}, inplace=True)

conc_df = pd.merge(feat_df, feat_complement_df, suffixes=('_sub', '_cmp'), validate='1:1', left_index=True, right_index=True)
conc_df = pd.merge(rr_df, conc_df, suffixes=('_rrdf', ''), validate='1:1', left_index=True, right_index=True)

def get_cache_label_indices(df):
    cache_size = int(df.shape[0]*(int(csz)/100))
    break_index = df.sort_values(by='TestViewCount', ascending=False).index[cache_size]
    return df['TestViewCount'] >= df.loc[break_index, 'TestViewCount']

c_lbl = get_cache_label_indices(conc_df)
conc_df['Y'] = np.where(c_lbl, 'sub', 'all')

print(conc_df.iloc[0])

conc_df.drop(columns=['query_sub', 'query_cmp'], inplace=True)
conc_df.rename(columns={csz: 'rr_sb', '100': 'rr_al'}, inplace=True)
conc_df.to_csv(ml_file, index=False)
print(list(conc_df.columns))

Query                     'Two Towers' Sets
TestViewCount                         49214
2                                       0.1
100                               0.0666667
query_sub                            sudoku
covered_t_sub                             0
covered_c_sub                             1
mean_df_t_sub                             0
mean_df_c_sub                   0.000384954
min_df_t_sub                              0
min_df_c_sub                    0.000384954
mean_mean_pop_t_sub                       0
mean_min_pop_t_sub                        0
min_mean_pop_t_sub                        0
min_min_pop_t_sub                         0
mean_mean_pop_c_sub                 68136.1
mean_min_pop_c_sub                     9821
min_mean_pop_c_sub                  68136.1
min_min_pop_c_sub                      9821
ql_t_sub                        1.83133e-07
ql_c_sub                        4.68542e-07
qll_t_sub                          -15.5131
qll_c_sub                       